In [2]:
import os
import uuid
import datetime
import pandas as pd
import numpy as np


dir = r'../../data'


def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir): 
        if "ok" in os.path.basename(root):
            for name in files:
                filepath = root + os.sep + name
                if filepath.endswith(".csv") and "2021" in filepath:
                    r.append(os.path.join(root, name))
                #end if
            #end for
    #end for
    return sorted(r)
#end list_files



In [3]:
def process_logs(df_name, weeks_to_keep, truncate_to_week):
    df = pd.read_csv(df_name, sep=",")
    df["date"] = pd.to_datetime(df['Hora'].str.zfill(16), dayfirst=True) #format='%d/%m/%y %H:%M')
    #df["date"] = df['date'] - pd.to_timedelta(df['date'].dt.dayofweek, unit='d') if truncate_to_week else df["date"]
    #eliminate hours, minutes and seconds
    df["date"] = df["date"].dt.date

    df["key"] = df["Nome completo"]
    #print("Unique users found: {}".format(df.key.nunique()))
    
    df = df.groupby([df['date'], df["key"]])['Nome do evento'].agg(['count']).reset_index().rename(columns={'count':'count_logs'})

    # date, key, count_logs
    # date(day), key, count_vpl, count_forum, count_url, count_system, count_page, count_file, count_user_report

    df["date"] = pd.to_datetime(df['date'])
    #fill count with 0s for all dates seen in the log given that the user has no log in that date
    df = df.set_index(['date', 'key']).unstack(fill_value=0).asfreq(
        'D', fill_value=0).stack().sort_index(level=1).reset_index()

    #get first date and add the offset
    thre = df.date.min() + datetime.timedelta(weeks = weeks_to_keep)
    #print(df.date.min())
    #print(thre)
    #filter out weeks out of range
    df = df[(df['date'] < thre)]

    freqq = "W" if truncate_to_week else "d"
    df = (df.groupby(['key', pd.Grouper(freq=freqq, key='date', closed='left')])
            ['count_logs'] #add new columns
            .sum()
            .unstack(fill_value=0)).reset_index()
    df.columns = df.columns.map(str)
    col_counter = 1
    new_col_names = []
    for colname in df.columns:
        if colname == "key":
            new_col_names.append(colname)
        else:
            new_col_names.append("datapoint{}".format(col_counter))   #gather suffix and append to datapoint-> "datapoint_"+colname.substring("_")+"{}"
            col_counter += 1                                          #datapoint_vpl1, datapoint_url1
        #end
    #end
    df.columns = new_col_names

    return df
#end process_logs

def process_grades(df_name):
    df = pd.read_csv(df_name, sep=",")
    df["key"] = df["Nome"] + " (" + df["Matrícula"].astype(str) + ")"
    df["target"] = df["Total do curso (Real)"].replace("-", 0.0).astype(float)
    df["target_cat"] = np.where(df['target'] >= 6.0 , 0, 1)
    return df[["key", "target", "target_cat"]]
#end process_grades

In [4]:
def process_dfs(df_list, weeks_to_keep=10, truncate_to_week=False):
    to_concat = []
    for i in range(0, len(df_list), 2):
        log_first = True if "logs" in df_list[i] else False
        turma = df_list[i].split(os.sep)[3]
        if log_first:
            #generate key for logs
            #generate key for grades
            #merge

            #change parameter to both functions to the merged df
            df_logs = process_logs(df_list[i], weeks_to_keep, truncate_to_week)
            df_grades = process_grades(df_list[i+1])
        else:
            df_grades = process_grades(df_list[i])
            df_logs = process_logs(df_list[i+1], weeks_to_keep, truncate_to_week)
        #end if
        data_cols = df_logs.columns.tolist()
        data_cols.remove("key")
        print("Logs size = {}".format(len(df_logs)))
        print("Grades size = {}".format(len(df_grades)))
        final_df = df_logs.merge(df_grades, on="key", how="inner")

        #final_df = final_df.set_index("id_group")

        final_df["group"] = turma

        #final_df.loc[:, "id_group"] = 1
        final_df["id_group"] = 1
        #final_df.loc[:, "id_subject"] = 1
        final_df["id_subject"] = 1

        #anonymize groups and subjects
        final_df.loc[:, "id_group"] = final_df.groupby("group").id_group.transform(lambda g: str(uuid.uuid4()))
        final_df.loc[:, "id_subject"] = final_df.groupby("key").id_subject.transform(lambda g: str(uuid.uuid4()))

        ##final_df["id_group"] = final_df.groupby("group").transform(lambda group: str(uuid.uuid4()))
        ##final_df["id_subject"] = final_df.groupby("key").transform(lambda group: str(uuid.uuid4()))

        to_keep = ["id_group", "id_subject", "key"] + data_cols + ["target", "target_cat"]
        to_concat.append( final_df[to_keep] )
    #end for
    return pd.concat(to_concat)
#end anonymize_dfs


In [5]:
df_list = list_files(dir)
df_final = process_dfs(df_list, weeks_to_keep=20)

Logs size = 40
Grades size = 26
Logs size = 28
Grades size = 19


In [6]:
df_final.head()

,id_group,id_subject,key,datapoint1,datapoint2,datapoint3,datapoint4,datapoint5,datapoint6,datapoint7,...,datapoint133,datapoint134,datapoint135,datapoint136,datapoint137,datapoint138,datapoint139,datapoint140,target,target_cat
0,db6ec63e-07e0-4dc6-beb7-604e835da7ac,a5326788-2dd3-4d86-8edc-a321daaf4d92,Andrio Vicente Maguerroski (21105785),0,11,2,0,0,0,0,...,0,1,0,0,0,3,1,0,1.5,1
1,db6ec63e-07e0-4dc6-beb7-604e835da7ac,490080fd-d980-4734-b0ae-452627f2d17e,Arthur Paulino Malgarisi Aguiar (21205133),0,0,0,0,0,0,0,...,0,4,4,3,6,0,1,10,7.0,0
2,db6ec63e-07e0-4dc6-beb7-604e835da7ac,fb83e35f-ec0d-4d42-a468-fd508223798e,Bruna Reginato Bocalon (21200544),3,4,4,0,0,0,0,...,0,45,25,3,0,22,0,0,7.5,0
3,db6ec63e-07e0-4dc6-beb7-604e835da7ac,412e5930-a047-42ad-be43-bc2d4ad7de5d,Eduardo Girardi Cesa (21100699),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,1
4,db6ec63e-07e0-4dc6-beb7-604e835da7ac,8de814e2-600a-4b5a-a5fa-a90f0b967775,Elvio Antonio Lessa (21203900),3,5,1,1,1,0,0,...,19,16,10,18,13,15,6,6,9.5,0


In [70]:
today_ts = datetime.datetime.now()
today_ts = today_ts.strftime("%d-%m-%Y-%H-%M")

df_final.to_csv("../../datasets/daily/processed_dataset_{}.csv".format(today_ts), sep=',', encoding='utf-8', index=False)

In [8]:
df_final.query("target < 1").shape

(14, 26)